In [14]:
from typing import Tuple
import pandas as pd
import numpy as np

# Preprocess the text
Eliminate numbers, punctuation signs

In [15]:
import re

def preprocessing(text:str) -> str:
    text = re.sub(r'[\t\n]', ' ', text) # remove tabs and newlines
    text = re.sub(r'[0-9][0-9]\.|[0-9]\.', ' ', text) # remove lists
    text = re.sub(r'\[[0-9]+\]', ' ', text) # remove citatinos - ex: [321]
    text = re.sub(r'\\', ' ', text) # remove '\' from word\'s
    text = re.sub(r'\[.+\]', ' ', text) # remove [word] - ex: [edit]
    text = re.sub(r'\(|\)', ' ', text) # remove ()
    text = re.sub(r'[=\-\+]+', ' ', text) # remove ()
    text = re.sub(r' +', ' ', text) # reduce spaces to one space
    text = re.sub(r'\.+', '.', text) # replace ... with .
    return text

# Add BERT specific tags [CLS][SEP]

In [16]:
def split_add_tags_text(text: str) -> str:
    final_article: str = "[CLS]"
    text: str = preprocessing(text)
    article: list[str] = text.split(". ")
    for sentence in article:
        final_article = f"{final_article} {sentence}. [SEP]"
    
    final_article = re.sub('\.\.', '.', final_article) # replace ... with .
    return final_article

# Truncate the text after proportion (128 start, 384 end)

In [17]:
def truncate_text(text: str, start: int, end: int) -> str:

    # text_cleaned: str = preprocessing(text)
    text_cleaned: str = split_add_tags_text(text) # just for bert
    
    words: list[str] = text_cleaned.split(" ")
    words: list[str] = words[:start] + words[-end:]

    text_truncated: str= " ".join(word for word in words)

    return text_truncated

In [4]:
df_raw = pd.read_csv("Raw_Dataset.csv")

In [18]:
def apply_fun(row) -> Tuple[str, str, int]:
    title_row: str = row['title']
    # title_cleaned: str = preprocessing(title_row)
    title_cleaned: str = split_add_tags_text(title_row) # just for bert

    text_row: str = row['text']
    text_cleaned: str = truncate_text(text_row, 42, 86)

    return [title_cleaned,  text_cleaned, row.label]

In [19]:
cleaned_data = df_raw.apply(apply_fun, axis=1, result_type='expand')

In [20]:
df_cleaned_data = pd.DataFrame(cleaned_data)
df_cleaned_data.columns = ['title', 'text', 'label']

In [21]:
df_cleaned_data["index"] = range(1, len(df_cleaned_data) + 1)
df_cleaned_data = df_cleaned_data.set_index("index")

In [22]:
df_cleaned_data.to_csv(f"../CleanedDataset/Cleaned_Data-{128}-bert.csv")

In [2]:
df_cleaned_data = pd.read_csv("Cleaned_Data.csv")

In [11]:
import pandas_tfrecords
pandas_tfrecords.pd2tf(df_cleaned_data, "Cleaned_Data.tfrecord")

2021-12-05 08:47:22.975555: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-05 08:47:22.975981: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-12-05 08:47:34.395154: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-05 08:47:34.396642: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-05 08:47:34.396899: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.s

## Data Understanding

In [7]:
np.mean([len(str(x).split(" ")) for x in df_cleaned_data["title"].values.tolist()])

12.777840936461294

In [8]:
np.mean([len(str(x).split(" ")) for x in df_cleaned_data["text"].values.tolist()])

409.03751625800857